In [3]:
import pandas as pd
import numpy as np
import networkx as nx



In [4]:
train_graph=nx.read_edgelist('Train_pos.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)

print(nx.info(train_graph))

Name: 
Type: DiGraph
Number of nodes: 1780753
Number of edges: 7550015
Average in degree:   4.2398
Average out degree:   4.2398


# Jaccard Distance

\begin{equation}
Jaccard Distance = \frac{|X\cap Y|}{|X \cup Y|} 
\end{equation}

In [5]:
def jaccard_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a)))==0 | len(set(train_graph.successors(b)))==0:
            return 0
        
        X=set(train_graph.successors(a))
        Y=set(train_graph.successors(b))
        
        nume=len(X.intersection(Y))
        dene=len(X.union(Y))
        
        j=nume/dene
    except:
        return 0
    return j

In [6]:
#function Testing

print(jaccard_for_followees(3,1698856))

0.2


In [7]:
def jaccard_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a)))==0 | len(set(train_graph.predecessors(b)))==0:
            return 0
        
        X=set(train_graph.predecessors(a))
        Y=set(train_graph.predecessors(b))
        
        nume=len(X.intersection(Y))
        dene=len(X.union(Y))
        
        j=nume/dene
        
    except:
        return 0
    
    return j
        
        

In [8]:
# function Testing 

print(jaccard_for_followers(1,123455))

0.0


# Cosine distance

\begin{equation}
CosineDistance = \frac{|X\cap Y|}{|X|\cdot|Y|} 
\end{equation}

In [9]:
def cosine_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a)))==0 | len(set(train_graph.successors(b)))==0:
            return 0
        
        
        X=set(train_graph.successors(a))
        Y=set(train_graph.successors(b))
        
        nume=len(X.intersection(Y))
        dene=math.sqrt(len(X)*len(Y))
        
        cos=nume/dene
        
        return cos
        
    except:
        return 0
    

In [10]:
# testing Function

print(cosine_for_followees(273084,1505602))

0


In [11]:
def cosine_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a)))==0 | len(set(train_graph.predecessors(b)))==0:
            return 0
        
        
        X=set(train_graph.predecessors(a))
        Y=set(train_graph.predecessors(b))
        
        nume=len(X.intersection(Y))
        dene=lath.sqrt(len(X)*len(Y))
        
        cos=nume/dene
        
        return cos
    except:
        return 0
    
    

In [12]:
# Testing Function 

print(cosine_for_followers(669354,1635354))

0


# Ranking Measures

## Page Ranking 

In [13]:
%%time

import os
import pickle

if not os.path.isfile("/Users/saisantosh/Desktop/Facebook/page_rank.p"):
    pr=nx.pagerank(train_graph,alpha=0.85)
    pickle.dump(pr,open("/Users/saisantosh/Desktop/Facebook/page_rank.p",'wb'))
else:
    pr=pickle.load(open("/Users/saisantosh/Desktop/Facebook/page_rank.p",'rb'))

CPU times: user 323 ms, sys: 150 ms, total: 473 ms
Wall time: 533 ms


In [14]:
print('min=',pr[min(pr,key=pr.get)])

min= 1.6556710759016465e-07


In [15]:
mean_pr=float(sum(pr.values()))/len(pr)
print(mean_pr)

5.615601939137468e-07


# Shortest path

In [16]:
def compute_shortest_path_length(a,b):
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p=nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p=nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1

In [17]:
# testing function

print(compute_shortest_path_length(77697,826021))

10


# Weakly connected component(Community)

In [18]:
%%time
# getting wealky connected component
wcc=list(nx.weakly_connected_components(train_graph))
def belongs_to_same_wcc(a,b):
    index = []
    if train_graph.has_edge(b,a):
        return 1
    if train_graph.has_edge(a,b):
            for i in wcc:
                if a in i:
                    index= i
                    break
            if (b in index):
                train_graph.remove_edge(a,b)
                if compute_shortest_path_length(a,b)==-1:
                    train_graph.add_edge(a,b)
                    return 0
                else:
                    train_graph.add_edge(a,b)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if a in i:
                    index= i
                    break
            if(b in index):
                return 1
            else:
                return 0

CPU times: user 11.9 s, sys: 917 ms, total: 12.8 s
Wall time: 12.8 s


# Adamic/Adar index

Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

In [19]:
#Adar index

def calc_adar_in(a,b):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(train_graph.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0


In [20]:
# is person was following back

def follows_back(a,b):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0

# Kartz Centrality

In [21]:
%%time
if not os.path.isfile("/Users/saisantosh/Desktop/Facebook/katz.p"):
    katz=nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
    pickle.dump(katz,open("/Users/saisantosh/Desktop/Facebook/katz.p",'wb'))
else:
    katz=pickle.load(open("/Users/saisantosh/Desktop/Facebook/katz.p",'rb'))

CPU times: user 332 ms, sys: 139 ms, total: 470 ms
Wall time: 533 ms


In [22]:
print('min',katz[min(katz,key=katz.get)])

min 0.0007313441005502577


In [23]:
mean_katz=float(sum(katz.values()))/len(katz)
print(mean_katz)

0.000748371130635992


# HITS score

In [24]:
if not os.path.isfile("/Users/saisantosh/Desktop/Facebook/hits.p"):
    hits=nx.hits(train_graph,max_iter=100,tol=1e-08, nstart=None,normalized=True)
    pickle.dump(hits,open("/Users/saisantosh/Desktop/Facebook/hits.p",'wb'))
else:
    hits=pickle.load(open("/Users/saisantosh/Desktop/Facebook/hits.p",'rb'))

# Preferential_attachement

In [25]:
def followee_pref_atta(a,b):
    try:
        X=len(set(train_graph.successors(a)))
        Y=len(set(train_graph.successors(b)))
        return(X*Y)
    except:
        return 0

In [26]:
#testing function
followee_pref_atta(3,1698856)

35

In [27]:
def follower_pref_atta(a,b):
    try:
        X=len(set(train_graph.predecessors(a)))
        Y=len(set(train_graph.predecessors(b)))
        return(X*Y)
    
    except:
        return 0

In [28]:
#testing Function
follower_pref_atta(3,1698856)

66

# Reading a sample of data from both train and test set

In [29]:
# X_train file
import random
if os.path.isfile("/Users/saisantosh/Desktop/Facebook/X_Train.csv"):
    filename="/Users/saisantosh/Desktop/Facebook/X_Train.csv"
    
    n_train=15100028
    s=100000
    skip_train=sorted(random.sample(range(1,n_train+1),n_train-s))

In [30]:
# X_test file

if os.path.isfile("/Users/saisantosh/Desktop/Facebook/X_Test.csv"):
    filename="/Users/saisantosh/Desktop/Facebook/X_Test.csv"
    
    n_test=3775006
    s=50000
    skip_test=sorted(random.sample(range(1,n_test+1),n_test-s))


In [31]:
print("NUmber of rows in the Train datafile:",n_train)
print("Number of rows in the test data file:",n_test)

NUmber of rows in the Train datafile: 15100028
Number of rows in the test data file: 3775006


In [32]:
df_final_train=pd.read_csv('X_Train.csv',skiprows=skip_train,names=["source_node","destination_node"])
df_final_train['indicator_link']=pd.read_csv('Y_Train.csv',skiprows=skip_train,names=['indicator_link'])
print("our train matrix size",df_final_train.shape)
df_final_train.head()

our train matrix size (100002, 3)


,source_node,destination_node,indicator_link
0,733963,1130271,1
1,304948,813518,1
2,337938,1349535,1
3,772221,1664389,1
4,570602,1284835,1


In [33]:

df_final_test = pd.read_csv('X_Test.csv', skiprows=skip_test, names=['source_node', 'destination_node'])
df_final_test['indicator_link'] = pd.read_csv('Y_Test.csv', skiprows=skip_test, names=['indicator_link'])
print("Our test matrix size ",df_final_test.shape)
df_final_test.head()

Our test matrix size  (50002, 3)


,source_node,destination_node,indicator_link
0,443463,12032,1
1,1086987,1126458,1
2,18307,360899,1
3,974895,1745548,1
4,1758615,642057,1


# Adding Features

In [34]:
#mapping jaccard followers to train and test data(1)

df_final_train["Jaccard_followers"]=df_final_train.apply(lambda row:jaccard_for_followers(row['source_node'],row['destination_node']),axis=1)
df_final_test["Jaccard_followers"]=df_final_test.apply(lambda row:jaccard_for_followers(row['source_node'],row['destination_node']),axis=1)


#mapping Jaccard_followees to train and test data(2)

df_final_train["Jaccard_followees"]=df_final_train.apply(lambda row:jaccard_for_followees(row['source_node'],row['destination_node']),axis=1)
df_final_test["Jaccard_followees"]=df_final_test.apply(lambda row:jaccard_for_followees(row['source_node'],row['destination_node']),axis=1)

#mapping Cosine_followers to Train and test data(3)

df_final_train["Cosine_followers"]=df_final_train.apply(lambda row:cosine_for_followers(row['source_node'],row['destination_node']),axis=1)
df_final_test["Cosine_followers"]=df_final_test.apply(lambda row:cosine_for_followers(row['source_node'],row['destination_node']),axis=1)

#mapping Cosine_followees to Train and test data(4)

df_final_train["Cosine_followees"]=df_final_train.apply(lambda row:cosine_for_followees(row['source_node'],row['destination_node']),axis=1)
df_final_test["Cosine_followees"]=df_final_train.apply(lambda row:cosine_for_followees(row['source_node'],row['destination_node']),axis=1)

# mapping Adar index on train and test data(5)

df_final_train['adar_index'] = df_final_train.apply(lambda row: calc_adar_in(row['source_node'],row['destination_node']),axis=1)
df_final_test['adar_index'] = df_final_test.apply(lambda row: calc_adar_in(row['source_node'],row['destination_node']),axis=1)

# maping followback on train and test data(6)

df_final_train['follows_back'] = df_final_train.apply(lambda row: follows_back(row['source_node'],row['destination_node']),axis=1)
df_final_test['follows_back'] = df_final_test.apply(lambda row: follows_back(row['source_node'],row['destination_node']),axis=1)

#mapping weekly connected component on train and test data(7)

df_final_train['same_comp'] = df_final_train.apply(lambda row: belongs_to_same_wcc(row['source_node'],row['destination_node']),axis=1)
df_final_test['same_comp'] = df_final_test.apply(lambda row: belongs_to_same_wcc(row['source_node'],row['destination_node']),axis=1)


#mapping shortest path on train and test data(8)

df_final_train['shortest_path'] = df_final_train.apply(lambda row: compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)
df_final_test['shortest_path'] = df_final_test.apply(lambda row: compute_shortest_path_length(row['source_node'],row['destination_node']),axis=1)


#mapping page rank to source and destination in train and test data
# if there is no value impute mean value of page rank

df_final_train["page_rank_s"]=df_final_train.source_node.apply(lambda x:pr.get(x,mean_pr))
df_final_train["page_rank_d"]=df_final_train.destination_node.apply(lambda x:pr.get(x,mean_pr))

df_final_test["page_rank_s"]=df_final_test.source_node.apply(lambda x:pr.get(x,mean_pr))
df_final_test["page_rank_d"]=df_final_test.source_node.apply(lambda x:pr.get(x,mean_pr))


# mapping katz centrality score for source and destination in train and test data
#if there is no value impute mean value of mean katz score

df_final_train['katz_s'] = df_final_train.source_node.apply(lambda x: katz.get(x,mean_katz))
df_final_train['katz_d'] = df_final_train.destination_node.apply(lambda x: katz.get(x,mean_katz))

df_final_test['katz_s'] = df_final_test.source_node.apply(lambda x: katz.get(x,mean_katz))
df_final_test['katz_d'] = df_final_test.destination_node.apply(lambda x: katz.get(x,mean_katz))


#Mapping HITS(hubs) for source and desitination nodes in train and test data
#if there is no value impute 0

df_final_train['hubs_s'] = df_final_train.source_node.apply(lambda x: hits[0].get(x,0))
df_final_train['hubs_d'] = df_final_train.destination_node.apply(lambda x: hits[0].get(x,0))

df_final_test['hubs_s'] = df_final_test.source_node.apply(lambda x: hits[0].get(x,0))
df_final_test['hubs_d'] = df_final_test.destination_node.apply(lambda x: hits[0].get(x,0))



#Mapping HITS(authorities) for source and desitination nodes in train and test data
#if there is no value impute 0

df_final_train['authorities_s'] = df_final_train.source_node.apply(lambda x: hits[1].get(x,0))
df_final_train['authorities_d'] = df_final_train.destination_node.apply(lambda x: hits[1].get(x,0))

df_final_test['authorities_s'] = df_final_test.source_node.apply(lambda x: hits[1].get(x,0))
df_final_test['authorities_d'] = df_final_test.destination_node.apply(lambda x: hits[1].get(x,0))


# Mapping Preferential attachement

df_final_train["P_A_followee"]=df_final_train.apply(lambda row:followee_pref_atta(row["source_node"],row["destination_node"]),axis=1)
df_final_test["P_A_followee"]=df_final_test.apply(lambda row:followee_pref_atta(row["source_node"],row["destination_node"]),axis=1)


df_final_train["P_A_followers"]=df_final_train.apply(lambda row:follower_pref_atta(row["source_node"],row["destination_node"]),axis=1)
df_final_test["P_A_followers"]=df_final_test.apply(lambda row:follower_pref_atta(row["source_node"],row["destination_node"]),axis=1)




In [35]:
# defining function for few features(9,10,11,12,13,14)

def compute_features(df_final):
    num_followers_s=[]
    num_followees_s=[]
    num_followers_d=[]
    num_followees_d=[]
    inter_followers=[]
    inter_followees=[]
    
    for i,row in df_final.iterrows():
        try:
            s1=set(train_graph.predecessors(row["source_node"]))
            s2=set(train_graph.successors(row["source_node"]))
        except:
            s1=set()
            s2=set()
        try:
            d1=set(train_graph.predecessors(row["destination_node"]))
            d2=set(train_graph.successors(row["destination_node"]))
        except:
            d1=set()
            d2=set()
        
        num_followers_s.append(len(s1))
        num_followees_s.append(len(s2))
        
        num_followers_d.append(len(d1))
        num_followees_d.append(len(d2))
        
        inter_followers.append(len(s1.intersection(d1)))
        inter_followees.append(len(s2.intersection(d2)))
        
        
    return num_followers_s, num_followers_d, num_followees_s, num_followees_d, inter_followers, inter_followees

In [36]:
#calling compute_features function

df_final_train["num_followers_s"],df_final_train["num_followers_d"],df_final_train["num_followees_s"],df_final_train["num_followees_d"],df_final_train["inter_followers"],df_final_train["inter_followees"]=compute_features(df_final_train)

df_final_test["num_followers_s"],df_final_test["num_followers_d"],df_final_test["num_followees_s"],df_final_test["num_followees_d"],df_final_test["inter_followers"],df_final_test["inter_followees"]=compute_features(df_final_test)




# Weight Features

In [37]:
weight_in={}
weight_out={}
for i in (train_graph.nodes()):
    s1=set(train_graph.predecessors(i))
    w_in=1.0/(np.sqrt(1+len(s1)))
    weight_in[i]=w_in
    
    s2=set(train_graph.successors(i))
    w_out=1.0/(np.sqrt(1+len(s2)))
    weight_out[i]=w_out
    
mean_weight_in=np.mean(list(weight_in.values()))
mean_weight_out=np.mean(list(weight_out.values()))

In [38]:
df_final_train['weight_in']=df_final_train.destination_node.apply(lambda x:weight_in.get(x,mean_weight_in))
df_final_train['weight_out']=df_final_train.source_node.apply(lambda x:weight_out.get(x,mean_weight_out))

df_final_test['weight_in']=df_final_test.destination_node.apply(lambda x: weight_in.get(x,mean_weight_in))
df_final_test['weight_out']=df_final_test.source_node.apply(lambda x: weight_out.get(x,mean_weight_out))


#some feature engineering few weight 
df_final_train['weight_f1'] = df_final_train.weight_in + df_final_train.weight_out
df_final_train['weight_f2'] = df_final_train.weight_in * df_final_train.weight_out
df_final_train['weight_f3'] = (2*df_final_train.weight_in + 1*df_final_train.weight_out)
df_final_train['weight_f4'] = (1*df_final_train.weight_in + 2*df_final_train.weight_out)

#some features engineerings on the in and out weights
df_final_test['weight_f1'] = df_final_test.weight_in + df_final_test.weight_out
df_final_test['weight_f2'] = df_final_test.weight_in * df_final_test.weight_out
df_final_test['weight_f3'] = (2*df_final_test.weight_in + 1*df_final_test.weight_out)
df_final_test['weight_f4'] = (1*df_final_test.weight_in + 2*df_final_test.weight_out)

# SVD

In [39]:
def svd(x, S):
    try:
        z = sadj_dict[x]
        return S[z]
    except:
        return [0,0,0,0,0,0]

In [40]:
#for svd features to get feature vector creating a dict node val and inedx in svd vector
sadj_col = sorted(train_graph.nodes())
sadj_dict = { val:idx for idx,val in enumerate(sadj_col)}

In [41]:
Adj = nx.adjacency_matrix(train_graph,nodelist=sorted(train_graph.nodes())).asfptype()

In [42]:
from scipy.sparse.linalg import svds, eigs
U, s, V = svds(Adj, k = 6)
print('Adjacency matrix Shape',Adj.shape)
print('U Shape',U.shape)
print('V Shape',V.shape)
print('s Shape',s.shape)

Adjacency matrix Shape (1780753, 1780753)
U Shape (1780753, 6)
V Shape (6, 1780753)
s Shape (6,)


In [43]:
df_final_train[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
    df_final_train.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    
df_final_train[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
    df_final_train.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    #===================================================================================================
    
df_final_train[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',]] = \
    df_final_train.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

df_final_train[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
    df_final_train.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)
    #===================================================================================================
    
df_final_test[['svd_u_s_1', 'svd_u_s_2','svd_u_s_3', 'svd_u_s_4', 'svd_u_s_5', 'svd_u_s_6']] = \
    df_final_test.source_node.apply(lambda x: svd(x, U)).apply(pd.Series)
    
df_final_test[['svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3', 'svd_u_d_4', 'svd_u_d_5','svd_u_d_6']] = \
    df_final_test.destination_node.apply(lambda x: svd(x, U)).apply(pd.Series)

    #===================================================================================================
    
df_final_test[['svd_v_s_1','svd_v_s_2', 'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6',]] = \
    df_final_test.source_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

df_final_test[['svd_v_d_1', 'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5','svd_v_d_6']] = \
    df_final_test.destination_node.apply(lambda x: svd(x, V.T)).apply(pd.Series)

In [ ]:
#storing new features into HDFS file


In [ ]:
df_final_train.columns

In [ ]:
from pandas import HDFStore
hdf=HDFStore("Storage_sample_stage.h5")
hdf.put("final_train",df_final_train,format='table',data_columns=True)
hdf.put("final_test",df_final_test,format='table',data_columns=True)
hdf.close()